In [1]:
// Start a testcontainer running a Postgres database
@file:DependsOn("org.testcontainers:junit-jupiter:1.21.3")
@file:DependsOn("org.testcontainers:postgresql:1.20.0")
import org.testcontainers.containers.PostgreSQLContainer
import org.testcontainers.utility.DockerImageName
import org.testcontainers.utility.MountableFile

val postgres = PostgreSQLContainer(DockerImageName.parse("postgres:16-alpine"))
    .withCopyToContainer(
        MountableFile.forHostPath("../src/test/resources/init_postgres.sql"),
        "/docker-entrypoint-initdb.d/init.sql"
    )
postgres.start()
DISPLAY("Is running: ${postgres.isRunning}")
DISPLAY("URL: ${postgres.getJdbcUrl()}")

Is running: true

URL: jdbc:postgresql://localhost:55063/test?loggerLevel=OFF

In [2]:
// Include the Spring Boot JDBC Client
@file:DependsOn("org.springframework.boot:spring-boot-starter-jdbc:4.0.0-M2")

In [3]:
import org.springframework.jdbc.core.JdbcTemplate

val dataSrc = createDataSource(
    jdbcUrl = postgres.jdbcUrl,
    username = "test",
    password = "test"
)

// Create Spring Boot Database Client
val client = JdbcTemplate(dataSrc)
client

org.springframework.jdbc.core.JdbcTemplate@eed99ce

In [4]:
import java.sql.ResultSet

// Use Spring JdbcTemplate to run your SQL commands
val result: List<String> = client.query("SELECT * FROM users") { rs: ResultSet, rowNum: Int ->
    "${rs.getInt(1)}: ${rs.getString(2)}"
}
result.joinToString("\n")

1: Alice
2: Bob
3: Charlie

In [5]:
// Remember to stop the container after use to prevent it from leaking
postgres.stop()